Halite is an online multiplayer game created by Two Sigma.  In the game, four participants command ships to collect an energy source called **halite**.  The player with the most halite at the end of the game wins.

As part the **[Halite competition](https://www.kaggle.com/c/halite/overview)**, you'll write your own intelligent bots to play the game.

In this tutorial, we'll explore the basics of gameplay, and you'll submit your first agent to the competition!

# Part 1: Get started

In this section, you'll learn more about how to play the game.

## Join the competition!

Begin by joining the competition.  Open a new window with the **[competition page](https://www.kaggle.com/c/halite/overview)**, and click on the **"Rules"** tab. 

<center>
<img src="https://i.imgur.com/KMOARGz.png" width=70%><br/>
</center>

This takes you to the rules acceptance page. You must accept the competition rules in order to participate. These rules govern how many submissions you can make per day, the maximum team size, and other competition-specific details. Click on **"I Understand and Accept"** to indicate that you will abide by the competition rules.

## Game rules 

In this section, we'll look more closely at the game rules and explore the different icons on the game board.

For context, we'll look at a game played by four random players.  You can use the animation below to view the game in detail: every move is captured and can be replayed. 

In [ ]:
#$HIDE$
!pip install 'kaggle-environments==0.3.9'
from kaggle_environments import make, evaluate
env = make("halite", debug=True)
env.run(["random", "random", "random", "random"])

In [ ]:
#$HIDE_INPUT$
env.render(mode="ipython", width=800, height=600)

The game is played in a 21 by 21 gridworld and lasts 400 timesteps.  Each player starts the game with 5,000 halite and one ship.  

Grid locations with **halite** are indicated by a light blue icon, where larger icons indicate more available halite.

<center>
<img src="https://i.imgur.com/3NENMos.png" width=50%><br/>
</center>

Players use **ships** to navigate the world and collect halite.  A ship can only collect halite from its current position.  When a ship decides to collect halite, it collects 25% of the halite available in its cell.  This collected halite is added to the ship's "cargo".  

<center>
<img src="https://i.imgur.com/eKN0kP3.png" width=50%><br/>
</center>

Halite in ship cargo is not counted towards final scores.  In order for halite to be counted, ships need to deposit their cargo into a **shipyard** of the same color.  A ship can deposit all of its cargo in a single timestep simply by navigating to a cell containing a shipyard.

<center>
<img src="https://i.imgur.com/LAc6fj8.png" width=50%><br/>
</center>

Players start the game with no shipyards.  To get a shipyard, a player must convert a ship into a shipyard, which costs 500 halite.  Also, shipyards can spawn (or create) new ships, which deducts 500 halite (per ship) from the player.

Two ships cannot successfully inhabit the same cell.  This event results in a **collision**, where:
- the ship with more halite in its cargo is destroyed, and 
- the other ship survives and instantly collects the destroyed ship's cargo.

<center>
<img src="https://i.imgur.com/BuIUPmK.png" width=50%><br/>
</center>

If you view the full game rules, you'll notice that there are more types of collisions that can occur in the game (for instance, ships can collide with enemy shipyards, which destroys the ship, the ship's cargo, and the enemy shipyard). 

In general, Halite is a complex game, and we have not covered all of the details here.  But even given these simplified rules, you can imagine that a successful player will have to use a relatively complicated strategy.  

## Game strategy

As mentioned above, a ship has two options at its disposal for collecting halite.  It can:
- collect (or mine) halite from its current position.
- collide with an enemy ship containing relatively more halite in its cargo.  In this case, the ship destroys the enemy ship and steals its cargo.

Both are illustrated in the figure below.  The "cargo" that is tracked in the player's scoreboard contains the total cargo, summed over all of the player's ships.
<center>
<img src="https://i.imgur.com/2DJX6Vt.png" width=70%><br/>
</center>

This raises some questions that you'll have to answer when commanding ships:
- Will your ships focus primarily on locating large halite reserves and mining them efficiently, while mostly ignoring and evading the other players?  
- Or, will you look for opportunities to steal halite from other players?  
- Alternatively, can you use a combination of those two strategies?  If so, what cues will you look for in the game to decide which option is best?  For instance, if all enemy ships are far away and your ships are located on cells containing a lot of halite, it makes sense to focus on mining halite.  Conversely, if there are many ships nearby with halite to steal (and not too much local halite to collect), it makes sense to attack the enemy ships.

You'll also have to decide how to control your shipyards, and how your ships interact with shipyards.  There are three primary actions in the game involving shipyards.  You can:
- convert a ship into a shipyard.  This is the only way to create a shipyard.  
- use a shipyard to create a ship.
- deposit a ship's cargo into a shipyard. 

These are illustrated in the image below.

<center>
<img src="https://i.imgur.com/fL5atut.png" width=70%><br/>
</center>

With more ships and shipyards, you can collect halite at a faster rate.  But each additional ship and shipyard costs you halite: how will you decide when it might be beneficial to create more?

# Part 2: Make a submission

In this section, you'll make your first submission to the competition.

## The notebook

The first thing to do is to create a Kaggle notebook where you'll store all of your code. 

Begin by clicking on the [**Notebooks tab**](https://www.kaggle.com/c/halite/notebooks) on the competition page. Then, click on **"New Notebook"**.

![](https://i.imgur.com/DHPyh7s.png)

Next, click on **"Create"**. (_Don't change the default settings: so, **"Python"** should appear under **"Select language"**, and you should have **"Notebook"** selected under **"Select type"**._)

![](https://i.imgur.com/qUVvr8k.png)

You now have a notebook where you'll develop your first agent!  If you're not sure how to use Kaggle Notebooks, we strongly recommend that you walk through **[this notebook](https://www.kaggle.com/alexisbcook/getting-started-with-titanic)** before proceeding.  It teaches you how to run code in the notebook.

## Your first agent

It's time to create your first agent!  Copy and paste the code in the cell below into your notebook.

In [ ]:
%%writefile submission.py

DIRS = ["NORTH", "SOUTH", "EAST", "WEST"]

def argmax(arr, key=None):
    return arr.index(max(arr, key=key)) if key else arr.index(max(arr))

def get_col_row(size, pos):
    return (pos % size, pos // size)

# This function will not hold up in practice
# E.g. cell getAdjacent(224) includes position 0, which is not adjacent
def get_to_pos(size, pos, direction):
    col, row = get_col_row(size, pos)
    if direction == "NORTH":
        return pos - size if pos >= size else size ** 2 - size + col
    elif direction == "SOUTH":
        return col if pos + size >= size ** 2 else pos + size
    elif direction == "EAST":
        return pos + 1 if col < size - 1 else row * size
    elif direction == "WEST":
        return pos - 1 if col > 0 else (row + 1) * size - 1

def getAdjacent(pos, size):
    return [
        get_to_pos(size, pos, "NORTH"),
        get_to_pos(size, pos, "SOUTH"),
        get_to_pos(size, pos, "EAST"),
        get_to_pos(size, pos, "WEST"),
    ]

def getDirTo(fromPos, toPos, size):
    fromY, fromX = divmod(fromPos, size)
    toY,   toX   = divmod(toPos,   size)
    if fromY < toY: return "SOUTH"
    if fromY > toY: return "NORTH"
    if fromX < toX: return "EAST"
    if fromX > toX: return "WEST"

# Each ship id will be assigned a state (one of COLLECT or DEPOSIT) which decides what it will do on a turn.
states = {}

def agent(obs, config):
    action = {}
    player_halite, shipyards, ships = obs.players[obs.player]
    size = config["size"]

    for uid, shipyard in shipyards.items():
        # Maintain one ship always
        if len(ships) == 0:
            action[uid] = "SPAWN"

    for uid, ship in ships.items():
        # Maintain one shipyard always
        if len(shipyards) == 0:
            action[uid] = "CONVERT"
            continue

        # If a ship was just made
        if uid not in states: states[uid] = "COLLECT"

        pos, halite = ship

        if states[uid] == "COLLECT":
            if halite > 500:
                states[uid] = "DEPOSIT"

            elif obs.halite[pos] < 100:
                best = argmax(getAdjacent(pos, size), key=obs.halite.__getitem__)
                action[uid] = DIRS[best]

        if states[uid] == "DEPOSIT":
            if halite < 200: states[uid] = "COLLECT"

            direction = getDirTo(pos, list(shipyards.values())[0], size)
            if direction: action[uid] = direction
            else: states[uid] = "COLLECT"
    return action

This is what the code cell does ...

The line `%%writefile submission.py` saves the agent to a file that we will submit to the competition.  

For now, run the next code cell to play your agent against three random agents.  Your agent is in the top left corner of the screen.

In [ ]:
from kaggle_environments import make, evaluate
env = make("halite", debug=True)
my_game = env.run(["submission.py", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)

Not bad!

## Submit to the competition

Now, it's time to submit your agent to the competition.  To do this, follow the instructions below:
#$SUBMIT_TO_COMP$

Go to **"My Submissions"** to view your score and episodes being played.

# Part 3: Keep learning!

This first agent is just the beginning.  Over time and with effort, your agents will gradually perform better and better.

- links to more agents. read them and learn from them
- links to notebooks tab in competition
- helper library
- full rules
- game ai course